In [43]:
catchment_name  = 'WykeBeck'

In [44]:
import sys
import os
import folium
import numpy.ma as ma
from folium import Map, FeatureGroup, LayerControl
import rasterio
from shapely.geometry import box
import geopandas as gpd
from pyproj import CRS
from rasterio.mask import mask
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import contextily as cx
import matplotlib.patches as mpatches
import rioxarray as rxr
from rasterio import plot
from shapely.geometry import mapping
import glob 

# Define whether to filter out values <0.1
remove_little_values = True

sys.path.append("../ProcessModelResults/")
from my_functions import *

# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    minx, miny, maxx, maxy = 437000,  426500,  445500, 434300
elif catchment_name == 'WykeBeck':
    minx, miny, maxx, maxy = 430004,  429978, 438660, 440996 
    
# Create a bounding box (this is used in preparing the rasters)
bbox = box(minx, miny, maxx, maxy)    

model_directory = '../../../FloodModelling/{}Models/'.format(catchment_name)

subcatchments = False

def save_clipped_to_gdf(raster, out_meta, catchment_gdf, fp):
    save_array_as_raster(raster, fp, out_meta)
    
    with rasterio.open(fp) as src:
        catchment_gdf=catchment_gdf.to_crs(src.crs)
        # print(Vector.crs)
        out_image, out_transform=mask(src,catchment_gdf.geometry,crop=True)
        out_meta=src.meta.copy() # copy the metadata of the source DEM

    out_meta.update({"driver":"Gtiff", "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })

    with rasterio.open(fp,'w',**out_meta) as dst:
        dst.write(out_image)    

### Read in main catchment boundary and add to boundaries dictionary

In [45]:
if catchment_name == 'LinDyke':
    catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/CatchmentLinDyke_exported.shp'.format(catchment_name))
elif catchment_name == 'WykeBeck':
    catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/WykeBeckCatchment.shp'.format(catchment_name))
    
# Add to dictionary    
boundaries_gpd_dict={}
boundaries_gpd_dict[catchment_name] =catchment_gdf

### Add subcatchment boundaries, if using

In [46]:
if subcatchments == True:
    for subcatchment in ['Garforth_East', 'Garforth_West', 'Kippax_West', 'South_Catchment', 'Kippax','Garforth', 'TB_South', 'TB_North']:
        # Read in boundary
        boundary = gpd.read_file(model_directory + "SubCatchmentBoundaries/{}/{}.shp".format(subcatchment, subcatchment))
        # Convert projection
        boundary = boundary.to_crs("epsg:27700") 
        # Add to dict
        boundaries_gpd_dict[subcatchment]=boundary

### Read in the land cover file, clip it, update some parameters, save to landcover directory 

In [51]:
landcover_clipped, out_meta = open_and_clip(model_directory + 'LandCoverData/{}.tif'.format(original_landcover_fp), bbox)
out_meta.update({"nodata":np.nan, "dtype" :'float64' })
landcover_clipped[landcover_clipped == -2147483648] = np.nan
save_array_as_raster(landcover_clipped, model_directory + 'LandCoverData/LandCover_clipped.tif', out_meta)

In [52]:
if catchment_name == 'LinDyke':
    original_landcover_fp = "LandCover_resized"
elif catchment_name == 'WykeBeck':
    original_landcover_fp = "LandCover_resized"

landcover_clipped, out_meta = open_and_clip(model_directory + 'LandCoverData/{}.tif'.format(original_landcover_fp), bbox)
out_meta.update({"nodata":np.nan, "dtype" :'float64' })
landcover_clipped[landcover_clipped == -2147483648] = np.nan
output_fp = model_directory + 'LandCoverData/LandCover_clipped.tif'
save_clipped_to_gdf(landcover_clipped, out_meta, catchment_gdf, output_fp)

In [53]:
classes_dict_lindyke = {'SubUrban':1, 'Arable':2, 'Deciduous woodland':3, 'Freshwater':4, 'Improved grassland':5, 'Urban':6, 
                  'Neutral grassland':7, 'Calcareous grassland':8, 'Heather grassland':9}

classes_dict_wykebeck = {'Arable':1, 'Urban':2, 'Freshwater':3, 'Deciduous woodland':4, 'SubUrban':5, 'Improved Grassland':6, 
                  'Neutral grassland':7, 'Calcareous grassland':8, 'Heather grassland':9}

classes_dicts = {'LinDyke':classes_dict_lindyke, 'WykeBeck':classes_dict_wykebeck}

### Create a modified version of the data with flag for urban (10) and non-urban (11), save to landcover directory

In [54]:
for landcover_class in  classes_dicts[catchment_name]:
    #print(landcover_class)
    # Get list of all classes
    ls_values = np.unique(landcover_clipped[~np.isnan(landcover_clipped)]).tolist()
    # Remove this class (so it is a list of all classes but this one)
    ls_values.remove(classes_dicts[catchment_name][landcover_class])
    # Set the value of this class to 10
    landcover_this_class = np.where(landcover_clipped==classes_dicts[catchment_name][landcover_class], 10, landcover_clipped)
    # Set all the rest to 11
    for i in ls_values:
        #print(i)
        landcover_this_class = np.where(landcover_this_class==i, 11, landcover_this_class)
        
    # Save
    fp_to_save =model_directory + f"LandCoverData/LandCover_{landcover_class}_classification.tif"
    save_clipped_to_gdf(landcover_this_class, out_meta, catchment_gdf, fp_to_save)    
    
    if landcover_class == 'Water':
        landcover_not_water = landcover_water.copy()
        landcover_not_water =  np.where(landcover_not_water==10, 12, landcover_not_water)
        landcover_not_water =  np.where(landcover_not_water==11, 10, landcover_not_water)
        landcover_not_water =  np.where(landcover_not_water==12, 11, landcover_not_water)

        fp_to_save= 'LandCoverData/LandCover_notwater_classification.tif'
        save_clipped_to_gdf(landcover_not_water, out_meta, catchment_gdf, fp_to_save)     

### Urban and suburban combo

In [55]:
ls_values = np.unique(landcover_clipped[~np.isnan(landcover_clipped)]).tolist()
ls_values.remove(classes_dicts[catchment_name]['Urban'])
ls_values.remove(classes_dicts[catchment_name]['SubUrban'])

# Convert the 3 to 10 (for water) and the rest to 11 (for non-water).  
landcover_urban_and_suburban = np.where(landcover_clipped==classes_dicts[catchment_name]['Urban'], 10, landcover_clipped)
landcover_urban_and_suburban = np.where(landcover_urban_and_suburban==classes_dicts[catchment_name]['SubUrban'], 10, landcover_urban_and_suburban)

# Convert the rest of the classes to 11
for i in ls_values:
    landcover_urban_and_suburban =  np.where(landcover_urban_and_suburban==i, 11, landcover_urban_and_suburban)

fp_to_save= model_directory + 'LandCoverData/LandCover_urban_and_suburban_classification.tif'
save_clipped_to_gdf(landcover_urban_and_suburban, out_meta, catchment_gdf, fp_to_save)      

### Create version where 10 means not water and 11 means water

### Trim landcover data to Garforth/Kippax boundary
(we need this for filtering the ..)

In [56]:
# for region_name in ['Kippax', 'Garforth', 'Kippax_West', 'Garforth_East', 'Garforth_West', 'South_Catchment', 
#                     'TB_South', 'TB_North']:
#     # Get region boundary
#     region_boundary = boundaries_gpd_dict[region_name]
        
#     # Trim land cover data
#     raster = rxr.open_rasterio(model_directory + "LandCoverData/LandCover_urban_classification2.tif", masked=True).squeeze()
#     raster_clipped = raster.rio.clip(region_boundary.geometry.apply(mapping),region_boundary.crs)
    
#     if not os.path.exists(model_directory + "LandCoverData/{}/".format(region_name)):
#         os.mkdir(model_directory + "LandCoverData/{}/".format(region_name))
#     raster_clipped.rio.to_raster(model_directory + "LandCoverData/{}/LandCover_urban_classification2.tif".format(region_name))
    
#     # Add to 
#     values,counts = np.unique(raster_clipped.data, return_counts=True)
#     if 10 not in values:
#         values = np.insert(values, 0, 10, axis=0)
#         counts = np.insert(counts, 0, 0, axis=0)
#     df[region_name] = counts

### Plot to check trimming process

In [57]:
# fig, axs =plt.subplots(ncols=7, figsize =(12,5))

# for num, region_name in enumerate(['Garforth_East', 'Garforth_West', 'Kippax_West', 'South_Catchment',  'Kippax', 'TB_South', 'TB_North']):
#     # Read in data
#     raster = rxr.open_rasterio(model_directory + "LandCoverData/{}/LandCover_urban_classification.tif".format(region_name), masked=True).squeeze()
#     raster.plot.imshow(ax=axs[num])
#     boundaries_gpd_dict[region_name] .plot(ax=axs[num], facecolor="none", edgecolor= 'black', linewidth=2)
#     axs[num].set_axis_off()
#     axs[num].legend('', frameon=False)

### Look at proportion of each which is urban
NB: NA refers to cells which are not within the boundary of the catchment (because we initially trimmed the catchment using the bounding box of the catchment)

In [58]:
# # The proportion of the area which is in within the catchment boundary
# for column in df.columns[1:]:
#     percent_urban = round((df[column][0]/ (df[column][1] + df[column][0]))*100,1)
#     print("% of {} which is urban: ".format(column), percent_urban)